In [1]:
# install missing packages
%pip install plotly
%pip install py-cpuinfo
%pip install pandas-datareader

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
# check system details
import os
import psutil
import cpuinfo

ram_info = psutil.virtual_memory()
print(f"Total RAM: {ram_info.total / 1024 / 1024 / 1024:.2f} GB")
print(f"Available RAM: {ram_info.available / 1024 / 1024 / 1024:.2f} GB")
print(f"Used RAM: {ram_info.used / 1024 / 1024 / 1024:.2f} GB")
print(f"Percentage Usage Of RAM: {ram_info.percent}%")
print(f"CPU Cores: {os.cpu_count()}")
print(f"CPU Speed: {cpuinfo.get_cpu_info()['hz_actual_friendly']}")
disk_info = psutil.disk_usage(os.getcwd())
print(f"Total Disk: {disk_info.total / 1024 / 1024 / 1024:.2f} GB")
print(f"Available Disk: {disk_info.free / 1024 / 1024 / 1024:.2f} GB")
print(f"Used Disk: {disk_info.used / 1024 / 1024 / 1024:.2f} GB")
print(f"Percentage Usage Of Disk: {disk_info.percent}%")

Total RAM: 15.47 GB
Available RAM: 14.12 GB
Used RAM: 1.10 GB
Percentage Usage Of RAM: 8.7%
CPU Cores: 4
CPU Speed: 2.5000 GHz
Total Disk: 24.99 GB
Available Disk: 16.92 GB
Used Disk: 8.07 GB
Percentage Usage Of Disk: 32.3%


In [3]:
# import requirements
import pandas as pd
from sklearn.metrics import mean_squared_error, r2_score
from nnet_regressor import Regression

2023-10-11 15:06:34.421576: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
# get the data
crime = pd.read_csv("crime_rate.csv")

In [5]:
# split up the data into training and testing
y = crime[["Crimes"]]
X = crime.drop(columns="Crimes")
trainX = X.head(int(0.8 * X.shape[0]))
trainy = y.head(int(0.8 * y.shape[0]))
testX = X.tail(int(0.2 * X.shape[0])).reset_index(drop=True)
testy = y.tail(int(0.2 * y.shape[0])).reset_index(drop=True)

In [6]:
# build the model
print("\n---- Crime Regression Analysis ----\n")
model = Regression(
    name="Tensorflow With Feature Engineering", 
    path=None,
    rename=False, 
    deep=True,
    time=False, 
    binary=True, 
    imputation=False, 
    variance=True,
    scale=True,
    atwood=True,
    binning=True,
    reciprocal=True, 
    interaction=True, 
    selection=True,
    plots=False,
)
try:
    model.load()  # load the machine learning pipeline
    predictions = model.predict(testX)
except:
    model.fit(trainX, trainy)  # build the machine learning pipeline
    predictions = model.predict(testX)
    # print(f"R2: {model.r2}")
    # print(f"RMSE: {model.rmse}")
    # print(f"In Control: {model.in_control}")


---- Crime Regression Analysis ----

Model Training:
> Transforming The Data
> Transforming Categorical Features
> Removing Constant Features
> Selecting Features
> Scaling Features
> Computing Atwoood Numbers
> Binning Features
> Computing Reciprocals
> Computing Interactions
> Removing Constant Features
> Selecting Features
> Scaling Features
> Training Neural Network
26.65 Minutes
Model Indicators:
> Perturbing Features
19.75 Minutes
Model Prediction:
> Transforming The New Data
> Getting Predictions
1.39 Seconds
Model Monitoring:
> Computing Feature Drift
8.91 Seconds


In [7]:
# model diagnostics
print("Model Indicators:")
for i, indicator in enumerate(model.indicators["Indicator"][:10]):
    print(f"{i+1}. {indicator}")
print(" ")
print("Feature Drift:")
for i, feature in enumerate(model.drift.loc[model.drift["pvalue"] < 0.05, "Feature"][:10]):
    print(f"{i+1}. {feature}")
if model.drift.loc[model.drift["pvalue"] < 0.05].shape[0] == 0:
    print("None")

Model Indicators:
1. Dc_Dist_23*CPI
2. Week_6*CPI
3. 1/Crimes(t-2)*1/PPI
4. Unemployment(0.2-0.4)*1/Crimes(t-2)
5. Dc_Dist_92*Crimes(t-2)
6. Year_2008*1/Crimes(t-1)
7. Year_2008*Crimes(t-1)
8. Dc_Dist_92*Crimes(t-4)
9. Year_2008*Crimes(t-4)
10. Dc_Dist_77*Crimes(t-2)
 
Feature Drift:
1. Dc_Dist_8*Crimes(t-1)
2. Dc_Dist_8*Crimes(t-2)
3. Dc_Dist_3*1/Unemployment
4. Dc_Dist_2*Unemployment
5. Dc_Dist_17*Unemployment
6. Dc_Dist_8*Unemployment
7. Dc_Dist_23*Unemployment
8. Dc_Dist_92*Unemployment
9. Dc_Dist_5*Unemployment
10. Dc_Dist_77*1/Unemployment


In [8]:
# score the model
rmse = mean_squared_error(
    y_true=testy.iloc[:,0].to_numpy(),
    y_pred=predictions,
    squared=False,
)
r2 = r2_score(
    y_true=testy.iloc[:,0].to_numpy(),
    y_pred=predictions,
)

print(f"RMSE: {rmse}")
print(f"R2: {r2}")

RMSE: 33.81761402524257
R2: 0.8434197474377729


In [9]:
# save the machine learning pipeline
model.dump()

In [10]:
# refit the model to include the test data
model.refit(testX, testy)

Model Retraining:
> Transforming The Updated Data
> Transforming Categorical Features
> Removing Constant Features
> Selecting Features
> Scaling Features
> Computing Atwoood Numbers
> Binning Features
> Computing Reciprocals
> Computing Interactions
> Removing Constant Features
> Selecting Features
> Scaling Features
> Training Neural Network


KeyboardInterrupt: 